In [ ]:
# Api Call Function
import http.client
page=1
def api_call(url, page):
    conn = http.client.HTTPSConnection("api.themoviedb.org")

    payload = "{}"
    url = f"/3/discover/movie?year=2018&page={page}&include_video=false&include_adult=false&sort_by=revenue.desc&region=US&language=en-US&api_key=69b1bbc66e37773ea1cc0568b2898163"
    conn.request("GET", url, payload)

    res = conn.getresponse()
    data = res.read()

    
    data1_dict=json.loads(data)
    
    return data1_dict

In [ ]:
# Parse Function
def parse_data(data_list):
    parsed_data_list = []

    for movie in data_list['results']:
        try:
            movie_tuple = (movie['id'], movie['title'], movie['release_date'], movie['genre_ids'][0], movie['popularity'], movie['vote_average'], movie['vote_count'])
            parsed_data_list.append(movie_tuple)
        except:
            movie_tuple = (movie['id'], movie['title'], movie['release_date'], 0, movie['popularity'], movie['vote_average'], movie['vote_count'])
            parsed_data_list.append(movie_tuple)
            
        
    return parsed_data_list 

In [ ]:
# Insert Function
def insert_into_db(cnx, cursor, parsed_data):
    stmt = "INSERT INTO data1 (movie_id, title, release_date, genre_ids, popularity, vote_avg, vote_count) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(stmt, parsed_data)
    cnx.commit()

In [ ]:
# Pagination Script
for i in range(1,320): 
    print(i)
    page = i
    movies = api_call(url, page)
    print(movies)
    parsed_data = parse_data(movies)
    insert_into_db(cnx, cursor, parsed_data)
    page+=1
    time.sleep(10)